함수호출

In [9]:
import pandas as pd
import time,datetime
import numpy as np
from pandas import *
from informs import portfolio
import class_informs_Copy4 as inform

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

import math
import random as ran
from scipy.stats import truncnorm
#import seaborn as sbn

import time,datetime
import cmath


import re
import matplotlib.pyplot as plt

import os
import ipyparallel as ipp

import generator
import discriminator

데이터 전처리

In [10]:
#timeseries data
all_data=pd.read_csv('data/Timeseries_data_SP500.csv')

all_data['DATE']=pd.to_datetime(all_data['DATE'])
all_data=all_data.rename(columns={'NAME':'name','DATE':'date','SEDOL':'sedol','SECTOR':'sector','BETA':'beta','ALPHA_SCORE':'as','BENCH_WEIGHT':'bw',"MCAP_Q":'mq'})
sedol_list=all_data['sedol'].unique().tolist()

date_list=list(set(list(all_data['date'])))
date_list.sort()


dic_data = {k: v for k, v in all_data.groupby('date')}


results_=pd.read_csv('data/results_template.csv')
results_=results_.rename(columns={'DATE':'date','SEDOL':'sedol','WEIGHTS':'w','FOUR_WEEKLY_RETURN':'r'})
results_['date']=pd.to_datetime(results_['date'])
dic_results_={k: v for k, v in results_.groupby('date')}


def preprocessing(date):
    
    using_dic = dic_data[pd.to_datetime(date)]
    using_dic['index']=list(using_dic.index)

    asset_list = []
    for i in using_dic["sedol"]:
        asset_list.append(i)


    dic_sedol_as = {using_dic["sedol"][i] : using_dic["as"][i] for i in using_dic.index}

    dic_bench = {using_dic["sedol"][i] : using_dic["bw"][i] for i in using_dic.index}

    dic_beta = {using_dic["sedol"][i] : using_dic["beta"][i] for i in using_dic.index}

    dic_sector = {using_dic["sector"][i] : [] for i in using_dic.index }

    dic_MCAP = {using_dic["mq"][i] : [] for i in using_dic.index}

    dic_aseet_MCAP = {using_dic["sedol"][i] : using_dic["mq"][i] for i in using_dic.index}
    
    dic_aseet_sector = {using_dic["sedol"][i] : using_dic["sector"][i] for i in using_dic.index}
    
    for i in using_dic.index:
        dic_sector[using_dic["sector"][i]].append(using_dic["sedol"][i])
        dic_MCAP[using_dic["mq"][i]].append(using_dic["sedol"][i])



    #risk:cov_mat
    date_str=str(date)[:10]
    risk_data=pd.read_csv('data/Riskmodels/cov_mat_%s.csv'%(date_str))
    risk_sedol=[]
    for i in risk_data['COLUMN_INDEX']:
#         print(i)
        if i in risk_sedol:
            break
        else:
            risk_sedol.append(i)
#     risk_sedol=risk_data['ROW_INDEX'].unique().tolist()
    risk_mat = np.zeros((len(risk_sedol),len(risk_sedol)))
    risk_mat[np.triu_indices(len(risk_sedol), 0)] = list(risk_data['VALUE'])
    irows,icols = np.triu_indices(len(risk_sedol),0)
    risk_mat[icols,irows]=risk_mat[irows,icols]
    #         return risk_data,risk_sedol,risk_mat

    
    alpha = []


    for i in risk_sedol:

        alpha.append(-1*dic_sedol_as[i])

    dic_r={dic_results_[date]["sedol"][i] : dic_results_[date]["r"][i] for i in using_dic.index}

    return using_dic,asset_list,dic_sedol_as,dic_bench,dic_beta,dic_sector,dic_MCAP,risk_sedol,risk_mat,dic_r,alpha,dic_aseet_MCAP,dic_aseet_sector

print("complete")

#generate direct view
rc = ipp.Client()
dv = rc[:]
# print(rc.ids)

dv.block=True
dv.execute('import numpy as np')
dv.execute('import class_informs as inform')
dv.execute('import time')

complete


<AsyncResult: execute:finished>

In [17]:
date=date_list[0]
using_dic,asset_list,dic_sedol_as,dic_bench,dic_beta,dic_sector,dic_MCAP,risk_sedol,risk_mat,dic_r,alpha,dic_aseet_MCAP,dic_aseet_sector=preprocessing(date)
print(risk_sedol[10])
print(risk_sedol[20])

print(risk_mat[3][20])

2795393
2779397
0.0286960929587


함수 정의

In [11]:
def cal_obj1(sol, risk_mat, as_) :

    a = np.full((len(risk_mat[0]), len(risk_mat[0])), 10000)
    risk_mat = risk_mat * a
    s_rr = np.dot(sol, risk_mat)
    s_r = np.dot(s_rr, sol)
    s_a = np.dot(sol, as_*10000)
    obj = s_r-s_a
    return obj

def cal_obj(sol, risk_mat, as_) :

    a = np.full((len(risk_mat[0]), len(risk_mat[0])), 10000)
    risk_mat = risk_mat * a
    sol_d = list(np.array(sol) - np.array(using_dic['bw']))
    s_rr = np.dot(sol_d, risk_mat)
    s_r = np.dot(s_rr, sol_d)
    s_a = np.dot(sol_d, as_*10000)
    obj = s_r-s_a
    return obj



def select_candi(sol) :
    
    diff_sub = np.array([0]*len(as_), float)
    diff_add = np.array([0]*len(as_), float)
    candi = []
    
#     if sum(sol) >= 50 :
#         for val in range(len(sol)) :
#             if sol[val] > 0. :
#                 candi.append(val)
        
#     else :

    for i in range(len(sol)) :
        if(sol[i] > 0) :
            w = sol[i]
            sol[i] = 0
            diff_sub[i] = cal_obj(sol, risk_mat, as_)
            if min(diff_sub[diff_sub>0]) == diff_sub[i] :
                for j in range(len(sol)) :
                    if sol[j] == 0 :
                        sol[j] = w
                        diff_add[j] = cal_obj(sol, risk_mat, as_)
                        sol[j] = 0
            sol[i] = w
        
    sol[np.where(diff_sub == min(diff_sub[diff_sub>0]))[0][0]] = 0
    sol[np.where(diff_add == min(diff_add[diff_add>0]))[0][0]] = 0.1

    for val in range(len(sol)) :
        if sol[val] > 0. :
            candi.append(val)
    print(candi)
    print("candi")

    return candi



def update(sol,lim_time,as_,w_pre,O_scale) :
    start_time = time.time()
    
    
    
    
    ex = inform.informs(10000, 10000, w_pre, O_scale,dic_sector,dic_bench,risk_sedol,dic_MCAP,dic_beta,alpha,risk_mat)
    ex.set_omega(risk_mat)

    new_list = []

    con = []
    nums = 0
    for k in risk_sedol :
        new_list.append(sol[k])
        if sol[k] == 1:
            con.append(nums)
        nums += 1
    result_list = []
    result_list2 = []
    
    
    
    print(con)
    print("con")
    while(True):

        if time.time() - start_time > lim_time:
            break
        
        ex.set_con(cons=con)
        new_list2 = []
        new_list3 = []
        try:
            aabbaa = ex.solve(0.01)
            
            
            if aabbaa[0] == 0:
                print("TE infeasible")
                TE_out = aabbaa[1]
                print("TE_out : ",TE_out)
                con = TE_change(sol,TE_out)
                sol = {}
                cheknumb = 0
                connum = 0
                for i in risk_sedol:
                    if connum < len(con):
                        if con[connum] == cheknumb:
                            sol.update({i:1})
                            connum += 1
                        else:
                            sol.update({i:0})
                    else:
                            sol.update({i:0})
                    cheknumb += 1
                
            else:
            
                if aabbaa[0] == "sector":
                    print("S infeasible")
                    con = Sector_change(sol,aabbaa[1],aabbaa[2])
                    cheknumb = 0
                    connum = 0
                    sol = {}
                    for i in risk_sedol:
                        if connum < len(con):
                            if con[connum] == cheknumb:
                                sol.update({i:1})
                                connum += 1
                            else:
                                sol.update({i:0})
                        else:
                                sol.update({i:0})
                        cheknumb += 1

                elif aabbaa[0] == "MCAPQ":
                    print("M infeasible")
                    con = MCAPQ_change(sol,aabbaa[1],aabbaa[2])
                    cheknumb = 0
                    connum = 0
                    sol = {}
                    for i in risk_sedol:
                        if connum < len(con):
                            if con[connum] == cheknumb:
                                sol.update({i:1})
                                connum += 1
                            else:
                                sol.update({i:0})
                        else:
                                sol.update({i:0})
                        cheknumb += 1

                else:
                    aaaa = aabbaa[0][0]
                    bbbb= aabbaa[0][1]
                    for k in risk_sedol :
                        new_list2.append(aaaa[k])
                    for k in risk_sedol :
                        new_list3.append(bbbb[k])
                    new_list = new_list2

                    result_list.append(aabbaa[0][2])
                    result_list2.append(aabbaa[0][0])
                    print("obj_val")
                    print(cal_obj1(new_list3, risk_mat, as_))
                    TE_out = aabbaa[1]
                    con = TE_change(sol,TE_out)
                    cheknumb = 0
                    connum = 0
                    sol = {}
                    for i in risk_sedol:
                        if connum < len(con):
                            if con[connum] == cheknumb:
                                sol.update({i:1})
                                connum += 1
                            else:
                                sol.update({i:0})
                        else:
                                sol.update({i:0})
                        cheknumb += 1
                    print("cplex_val")
                    print(aabbaa[0][2])
                

        except:
            print("cplex no solution")
            sc = Sector_check(sol)
            if sc == 100:
                mc = MCAPQ_check(sol)
                if mc == 100:
                    con = select_candi(new_list)
                else:
                    con = MCAPQ_change(sol,mc[0],mc[1])                    
            else:
                con = Sector_change(sol,sc[0],sc[1])
                
            
            cheknumb = 0
            connum = 0
            sol = {}
            for i in risk_sedol:
                if connum < len(con):
                    if con[connum] == cheknumb:
                        sol.update({i:1})
                        connum += 1
                    else:
                        sol.update({i:0})
                else:
                        sol.update({i:0})
                cheknumb += 1



    return result_list,result_list2

def update2(sol,lim_time,as_,w_pre,O_scale) :
    start_time = time.time()
    
    
    
    
    ex = inform.informs(10000, 10000, w_pre, O_scale,dic_sector,dic_bench,risk_sedol,dic_MCAP,dic_beta,alpha,risk_mat)
    ex.set_omega(risk_mat)

    new_list = []

    con = []
    nums = 0
    for k in risk_sedol :
        new_list.append(sol[k])
        if sol[k] == 1:
            con.append(nums)
        nums += 1
    result_list = []
    result_list2 = []
    
    
    
    print(con)
    print("con")
    while(True):

        if time.time() - start_time > lim_time:
            break
        
        ex.set_con(cons=con)
        new_list2 = []
        new_list3 = []

        aabbaa = ex.solve(0.01)


        if aabbaa != 0:
            aaaa = aabbaa[0]
            bbbb= aabbaa[1]
            for k in risk_sedol :
                new_list2.append(aaaa[k])
            for k in risk_sedol :
                new_list3.append(bbbb[k])
            new_list = new_list2

            result_list.append(aabbaa[2])
            result_list2.append(aabbaa[0])
            con = select_candi(new_list)


        else:
            print("TE infeasible")
            con = select_candi(new_list)





    return result_list,result_list2



def GAN(rotation,w_pre_gan,n):
    X_dim = nbasset
    HL_dim = 50
    Z_dim = 10

    np.random.seed()

    G = generator.Generator(Z_dim, HL_dim, X_dim)

    sample_D = discriminator.Discriminator(in_dim=X_dim, train=False, risk_mat=risk_mat, bw=bw, beta=beta, alpha=alpha, w_11=0.1,w_pre_gan=w_pre_gan,MONTH=n)
    train_D = discriminator.Discriminator(in_dim=X_dim, train=True, risk_mat=risk_mat, bw=bw, beta=beta, alpha=alpha, w_11=0.1,w_pre_gan=w_pre_gan,MONTH=n)

    G_solver = optim.Adam(G.parameters(), lr=2e-3)

    mb_size = 128

    best_obj = 100000.0
    best_sols = []

    losses = []

    start_time = time.time()

    feasible_threshold = 0.95

    adaptive_inf_scale = float(nbasset)

    for it in range(rotation):
        if it % 900 == 0:
            sample_noise = sample_Z(128, Z_dim)
            samples = G(sample_noise)

            sample_D.Zstar = best_obj
            sample_D.inf_scale = adaptive_inf_scale

            sample_D_values = sample_D(samples).data.numpy()
            sample_obj_values = sample_D.objs.data.numpy()

            fea_sample_obj_values = sample_obj_values[sample_D_values > feasible_threshold]
            sols = samples.data.numpy()[sample_D_values > feasible_threshold]

            if len(fea_sample_obj_values > 0):
                best_idx = np.argmin(fea_sample_obj_values)
                if np.min(fea_sample_obj_values) < best_obj:
                    best_obj = np.minimum(fea_sample_obj_values[best_idx], best_obj)
                    best_sols.append(sols[best_idx])

        z = Variable(torch.randn(mb_size, Z_dim))

        train_D.Zstar = best_obj
        train_D.inf_scale = adaptive_inf_scale

        G_loss = -torch.mean(torch.log(train_D(G(z))))

        G.zero_grad()
        G_loss.backward()
        G_solver.step()
        
#     sam=[]
#     for i in range(50):
#         n = sample_Z(128, 10)
#         samples = G(n)
#         sam.append(samples)
        
    return best_sols, sols,fea_sample_obj_values

        

def GAN_reulst(best_sols, sols,fea_sample_obj_values):    
    good_sols=np.argsort(fea_sample_obj_values)[:10]
    card_list=[list(np.where(sols[i]>0.001)[0]) for i in range(len(good_sols))]
    card_freq=np.zeros(nbasset)
    for i in range(len(card_list)):
        for j in card_list[i]:
            card_freq[j]+=1
    best=np.argsort(sum(sols))[::-1][:len(card_freq[card_freq>len(card_list)-5])]
    bestsol=np.where(best_sols[0]>0.001)[0]
    return best,bestsol

def init_cplex(risk_mat,risk_sedol):
    sedol_var_list =[]
    for i in risk_sedol:
        sedol_var_list.append("d"+str(i))

    for i in risk_sedol:
        sedol_var_list.append("q"+str(i))

    sedol_var_list.append("assum")
    #global alpha
    alpha = []
    for i in risk_sedol:
        alpha.append(-10000.*dic_sedol_as[i])

    qmat=[]
    for i in range(len(risk_mat)):
        qmat_1=[]
        qmat_1.append(sedol_var_list)
        new_risk_mat=[]
        for j in risk_mat[i]:
            new_risk_mat.append(20000*j)
        for j in risk_mat[i]:
            new_risk_mat.append(0)
        new_risk_mat.append(0)
        qmat_1.append(new_risk_mat)
        qmat.append(qmat_1)

    for i in range(len(risk_mat)):
        qmat_1=[]
        qmat_1.append(sedol_var_list)
        new_risk_mat=[]
        for j in risk_mat[i]:
            new_risk_mat.append(0)
        for j in risk_mat[i]:
            new_risk_mat.append(0)
        new_risk_mat.append(0)
        qmat_1.append(new_risk_mat)
        qmat.append(qmat_1)

    for i in range(1):
        qmat_1=[]
        qmat_1.append(sedol_var_list)
        new_risk_mat=[]
        for j in risk_mat[i]:
            new_risk_mat.append(0)
        for j in risk_mat[i]:
            new_risk_mat.append(0)
        new_risk_mat.append(0)
        qmat_1.append(new_risk_mat)
        qmat.append(qmat_1)

    q_con1 = []
    q_con2 = []
    q_val = []


    for i in range(len((risk_mat[0]))):
        for j in range(len(risk_mat[0])):
            if j >= i:
                q_con1.append(i)
                q_con2.append(j)
                if i == j:
                    ex_list = list(risk_mat[i])
                    q_val.append(0.5*ex_list[j]*10000)
                else:
                    ex_list = list(risk_mat[i])
                    q_val.append(ex_list[j]*10000)

    Q_con = []
    Q_con.append(q_con1)
    Q_con.append(q_con2)
    Q_con.append(q_val)
    
    return Q_con, qmat, alpha

print("complete")

complete


In [17]:
w_pre
for i in 


{'2000019': 0.0,
 '2002305': 0.0,
 '2002479': 0.0,
 '2005973': 0.0,
 '2007849': 0.0,
 '2008154': 0.0,
 '2011602': 0.0,
 '2017213': 0.0,
 '2018175': 0.0,
 '2019479': 0.0,
 '2019952': 0.0,
 '2020459': 0.0,
 '2021690': 0.0,
 '2023607': 0.0,
 '2023737': 0.0,
 '2023748': 0.0,
 '2024020': 0.0,
 '2026082': 0.0,
 '2026242': 0.0,
 '2026361': 0.0,
 '2027104': 0.0,
 '2027342': 0.0,
 '2028572': 0.0,
 '2029809': 0.0,
 '2032067': 0.0,
 '2032380': 0.0,
 '2033004': 0.0,
 '2034494': 0.0,
 '2034524': 0.0,
 '2038430': 0.0,
 '2039831': 0.0,
 '2041364': 0.0,
 '2043694': 0.0,
 '2043962': 0.0,
 '2044545': 0.0,
 '2046251': 0.0,
 '2046552': 0.0,
 '2047317': 0.0,
 '2048804': 0.0,
 '2050832': 0.0,
 '2057059': 0.0,
 '2065159': 0.0,
 '2065308': 0.0,
 '2065955': 0.0,
 '2066408': 0.0053726894048245159,
 '2066721': 0.0,
 '2068921': 0.0,
 '2069065': 0.0049569062518690634,
 '2073022': 0.0,
 '2073408': 0.012401710806552554,
 '2077905': 0.0,
 '2081779': 0.00279885795381008,
 '2084637': 0.0,
 '2085102': 0.0,
 '2085878': 0

In [12]:
def TE_change(w_dic,TE_out):
    
    TE_in = -999
    CH_be = 99999
    nnn = 0
    candi = []
    
    ch_list = []
    for i in w_dic:
        if w_dic[i] == 0:
            ch_list.append(abs(dic_bench[i]*dic_bench[i]*risk_mat[nnn][nnn]))

    
    
    
    
    for i in w_dic:
        if w_dic[i] == 0:
            if CH_be > abs(max(ch_list)/2-abs(dic_bench[i]*dic_bench[i]*risk_mat[nnn][nnn])):
                CH_be = abs(max(ch_list)/2-abs(dic_bench[i]*dic_bench[i]*risk_mat[nnn][nnn]))
                TE_in = nnn
        nnn += 1
        
    w_list = []
    
    for i in risk_sedol:
        w_list.append(w_dic[i])
    
    
    w_list[TE_out] = 0
    w_list[TE_in] = 0.1

    for val in range(len(w_list)) :
        if w_list[val] > 0. :
            candi.append(val)
    print(candi)
    #print("candi")

    return candi
# date=date_list[0]
# using_dic,asset_list,dic_sedol_as,dic_bench,dic_beta,dic_sector,dic_MCAP,risk_sedol,risk_mat,dic_r,alpha,dic_aseet_MCAP,dic_aseet_sector=preprocessing(date)

# as_ = using_dic['as']
# sol = {}
# indnummm = 0

# for i in risk_sedol:
#     if indnummm < 70:
#         sol.update({i:1})
#     else:
#         sol.update({i:0})
#     indnummm += 1
    

# aaaaaaaaaaa = 0
    
# for j in range(10):
#     con = TE_change(sol,aaaaaaaaaaa)
#     aaaaaaaaaaa += 1
#     cheknumb = 0
#     connum = 0
#     sol = {}
#     for i in risk_sedol:
#         if connum < len(con):
#             if con[connum] == cheknumb:
#                 sol.update({i:1})
#                 connum += 1
#             else:
#                 sol.update({i:0})
#         else:
#                 sol.update({i:0})
#         cheknumb += 1    
    
    

In [13]:
def MCAPQ_check(w_dic):

    MCAPQ_sum_dic = {}
    d_sum_dic = {}
    d_sum_dic2 = {}
    #date=date_list[0]
    #using_dic,asset_list,dic_sedol_as,dic_bench,dic_beta,dic_sector,dic_MCAP,risk_sedol,risk_mat,dic_r,alpha,dic_aseet_MCAP=preprocessing(date)

    for i in dic_MCAP:
#         updatenum = -0.1
        d_sum_dic.update({i:0})
        d_sum_dic2.update({i:0})
#         for j in dic_sector[i]:
#             updatenum += dic_bench[j]
#         Sector_sum_dic.update({i:updatenum})

    #print(Sector_sum_dic)

    for i in risk_sedol:
        if w_dic[i] > 0:
            savenum = d_sum_dic[dic_aseet_MCAP[i]]
            savenum += 0.05
            d_sum_dic.update({dic_aseet_MCAP[i]:savenum})
            savenum = 0
        else:
            savenum = d_sum_dic[dic_aseet_MCAP[i]]
            savenum = savenum - dic_bench[i]
            d_sum_dic.update({dic_aseet_MCAP[i]:savenum})
            savenum = 0
    
    
              
                                

                            
                            
            
    in_MCAPQ = -999
    out_MCAPQ = -999
    for i in d_sum_dic:
        if d_sum_dic[i] < -0.01:
            in_MCAPQ = i
            print("IN_MCAPQ :",in_MCAPQ)
            print(d_sum_dic[i],"1111111111111111")
            print(d_sum_dic)
            out_MCAPQ = max(d_sum_dic.keys(), key=(lambda k: d_sum_dic[k]))
            print("out : ",out_MCAPQ) 
            return in_MCAPQ,out_MCAPQ
            break

    
    
        
        
        
    if in_MCAPQ == -999:
#         for i in risk_sedol:
#             if w_dic[i] > 0:
#                 savenum = d_sum_dic2[dic_aseet_MCAP[i]]
#                 if 0.01 - dic_bench[i] > -0.05:
#                     savenum = savenum + 0.01 - dic_bench[i]
#                     d_sum_dic2.update({dic_aseet_MCAP[i]:savenum})
#                     savenum = 0
#                 else:
#                     savenum = savenum -0.05
#                     d_sum_dic2.update({dic_aseet_MCAP[i]:savenum})
#                     savenum = 0
#             else:
#                 savenum = d_sum_dic2[dic_aseet_MCAP[i]]
#                 savenum = savenum - dic_bench[i]
#                 d_sum_dic2.update({dic_aseet_MCAP[i]:savenum})
#                 savenum = 0
        
#         for i in d_sum_dic2:
#             if d_sum_dic2[i] > 0.01:
#                 out_MCAPQ = i
#                 in_MCAPQ = min(d_sum_dic2.keys(), key=(lambda k: abs(d_sum_dic2[k])))
#                 print(d_sum_dic2[i],"22222222222222222222222")
#                 print("IN_MCAPQ : ",in_MCAPQ)
#                 print("out : ",out_MCAPQ)
#                 return in_MCAPQ,out_MCAPQ
#                 break
        print("nono MC")
        return 100   
#     if in_MCAPQ == -999:
        
    

def MCAPQ_change(sol,inM,outM):
    w_list = []
    
    for i in risk_sedol:
        w_list.append(sol[i])


    print("inM : ",inM)
    print("outM : ",outM)
    
    diff_sub = np.array([0]*len(as_), float)
    diff_add = np.array([0]*len(as_), float)
    save_sub = np.array([0]*len(as_), float)
    save_add = np.array([0]*len(as_), float)
    candi = []
    indnum = 0
    indnum2 = 0
    
    for i in risk_sedol :
        if sol[i] > 0 and dic_aseet_MCAP[i] == outM:
            w = sol[i]
            save_sub[indnum] = w
            diff_sub[indnum] = cal_obj1(save_sub, risk_mat, as_)

            if max(diff_sub[diff_sub>0]) == diff_sub[indnum] :
                for j in risk_sedol :
                    if sol[j] == 0 and dic_aseet_MCAP[j] == inM:
                        save_add[indnum2] = w
                        diff_add[indnum2] = cal_obj1(save_add, risk_mat, as_)
                        #print(diff_add[indnum2])
                        save_add[indnum2] = 0
                    indnum2 += 1
            sol[i] = w
            save_sub[indnum]=0
        indnum += 1
        indnum2 = 0
    #print(diff_sub)
    #print(w_list[np.where(diff_sub == min(diff_sub[diff_sub>0]))[0][0]])
    print(np.where(diff_sub == min(diff_sub[diff_sub>0]))[0][0])
    
#     print(w_list[np.where(diff_add == min(diff_add[diff_add>0]))[0][0]])
    print(np.where(diff_add == min(diff_add[diff_add>0]))[0][0])
    w_list[np.where(diff_sub == min(diff_sub[diff_sub>0]))[0][0]] = 0
    w_list[np.where(diff_add == min(diff_add[diff_add>0]))[0][0]] = 0.1

    for val in range(len(w_list)) :
        if w_list[val] > 0. :
            candi.append(val)
    print(candi)
    #print("candi")

    return candi

# date=date_list[0]
# using_dic,asset_list,dic_sedol_as,dic_bench,dic_beta,dic_sector,dic_MCAP,risk_sedol,risk_mat,dic_r,alpha,dic_aseet_MCAP,dic_aseet_sector=preprocessing(date)

# as_ = using_dic['as']
# sol = {}
# indnummm = 0

# for i in risk_sedol:
#     if indnummm < 70:
#         sol.update({i:1})
#     else:
#         sol.update({i:0})
#     indnummm += 1
    

# for j in range(10):
#     con = MCAPQ_change(sol)
#     cheknumb = 0
#     connum = 0
#     sol = {}
#     for i in risk_sedol:
#         if connum < len(con):
#             if con[connum] == cheknumb:
#                 sol.update({i:1})
#                 connum += 1
#             else:
#                 sol.update({i:0})
#         else:
#                 sol.update({i:0})
#         cheknumb += 1


In [14]:
def Sector_check(w_dic):

    Sector_sum_dic = {}
    d_sum_dic = {}
    d_sum_dic2 = {}
    #date=date_list[0]
    #using_dic,asset_list,dic_sedol_as,dic_bench,dic_beta,dic_sector,dic_MCAP,risk_sedol,risk_mat,dic_r,alpha,dic_aseet_MCAP=preprocessing(date)

    for i in dic_sector:
#         updatenum = -0.1
        d_sum_dic.update({i:0})
        d_sum_dic2.update({i:0})
#         for j in dic_sector[i]:
#             updatenum += dic_bench[j]
#         Sector_sum_dic.update({i:updatenum})

    #print(Sector_sum_dic)

    for i in risk_sedol:
        if w_dic[i] > 0:
            savenum = d_sum_dic[dic_aseet_sector[i]]
            savenum += 0.05
            d_sum_dic.update({dic_aseet_sector[i]:savenum})
            savenum = 0
        else:
            savenum = d_sum_dic[dic_aseet_sector[i]]
            savenum = savenum - dic_bench[i]
            d_sum_dic.update({dic_aseet_sector[i]:savenum})
            savenum = 0
    
            
    in_Sector = -999
    out_Sector = -999
    for i in d_sum_dic:
        if d_sum_dic[i] < -0.01:
            in_Sector = i
            print(d_sum_dic[i])
            print(d_sum_dic)
            out_Sector = max(d_sum_dic.keys(), key=(lambda k: d_sum_dic[k]))
            print("in_Sector : ",in_Sector,"  out_Sector : ",out_Sector)
            return in_Sector,out_Sector
            break
    
    
        
        
        
    if in_Sector == -999:
#         for i in risk_sedol:
#             if w_dic[i] > 0:
#                 savenum = d_sum_dic2[dic_aseet_sector[i]]
#                 if 0.01 - dic_bench[i] > -0.05:
#                     savenum = savenum + 0.01 - dic_bench[i]
#                     d_sum_dic2.update({dic_aseet_sector[i]:savenum})
#                     savenum = 0
#                 else:
#                     savenum = savenum -0.05
#                     d_sum_dic2.update({dic_aseet_sector[i]:savenum})
#                     savenum = 0
#             else:
#                 savenum = d_sum_dic2[dic_aseet_sector[i]]
#                 savenum = savenum - dic_bench[i]
#                 d_sum_dic2.update({dic_aseet_sector[i]:savenum})
#                 savenum = 0
        
#         for i in d_sum_dic2:
#             if d_sum_dic2[i] > 0.01:
#                 out_Sector = i
#                 in_Sector = min(d_sum_dic2.keys(), key=(lambda k: abs(d_sum_dic2[k])))
#                 print(d_sum_dic2[i],"2222222222222222222222222")
#                 print("in_Sector : ",in_Sector,"  out_Sector : ",out_Sector)
#                 return in_Sector,out_Sector
#                 break
        print("nono se")
        return 100  
#     if in_Sector == -999:
        
    

def Sector_change(sol,inS,outS):
    w_list = []
    
    for i in risk_sedol:
        w_list.append(sol[i])


    print("inS : ",inS)
    print("outS : ",outS)
    
    diff_sub = np.array([0]*len(as_), float)
    diff_add = np.array([0]*len(as_), float)
    save_sub = np.array([0]*len(as_), float)
    save_add = np.array([0]*len(as_), float)
    candi = []
    indnum = 0
    indnum2 = 0
    
    for i in risk_sedol :
        if sol[i] > 0 and dic_aseet_sector[i] == outS:
            w = sol[i]
            save_sub[indnum] = w
            diff_sub[indnum] = cal_obj1(save_sub, risk_mat, as_)

            if max(diff_sub[diff_sub>0]) == diff_sub[indnum] :
                for j in risk_sedol :
                    if sol[j] == 0 and dic_aseet_sector[j] == inS:
                        save_add[indnum2] = w
                        diff_add[indnum2] = cal_obj1(save_add, risk_mat, as_)
                        #print(diff_add[indnum2])
                        save_add[indnum2] = 0
                    indnum2 += 1
            sol[i] = w
            save_sub[indnum]=0
        indnum += 1
        indnum2 = 0
    #print(diff_sub)
    #print(w_list[np.where(diff_sub == min(diff_sub[diff_sub>0]))[0][0]])
    print(np.where(diff_sub == min(diff_sub[diff_sub>0]))[0][0])
    
#     print(w_list[np.where(diff_add == min(diff_add[diff_add>0]))[0][0]])
    print(np.where(diff_add == min(diff_add[diff_add>0]))[0][0])
    w_list[np.where(diff_sub == min(diff_sub[diff_sub>0]))[0][0]] = 0
    w_list[np.where(diff_add == min(diff_add[diff_add>0]))[0][0]] = 0.1

    for val in range(len(w_list)) :
        if w_list[val] > 0. :
            candi.append(val)
    print(candi)
    #print("candi")

    return candi

# date=date_list[0]
# using_dic,asset_list,dic_sedol_as,dic_bench,dic_beta,dic_sector,dic_MCAP,risk_sedol,risk_mat,dic_r,alpha,dic_aseet_MCAP,dic_aseet_sector=preprocessing(date)

# as_ = using_dic['as']
# sol = {}
# indnummm = 0

# for i in risk_sedol:
#     if indnummm < 70:
#         sol.update({i:1})
#     else:
#         sol.update({i:0})
#     indnummm += 1
    

# for j in range(10):
#     con = Sector_change(sol)
#     cheknumb = 0
#     connum = 0
#     sol = {}
#     for i in risk_sedol:
#         if connum < len(con):
#             if con[connum] == cheknumb:
#                 sol.update({i:1})
#                 connum += 1
#             else:
#                 sol.update({i:0})
#         else:
#                 sol.update({i:0})
#         cheknumb += 1


solve!

In [16]:
#for i in date: 이런식으로 하되 evaluation criteria 는 아직 반영안함..!
feasi = 0
infeasi = 0
list_feasi_obj = []
n = 0
fini = True
k = 0
w_pre = {}
w_pre_list = []
############################################
while(fini):

    
    print(n, "period")

    date=date_list[n]
    init_set='random' #initial set by GAN, cplex, random

    c_time = 10. #cplex time for initial set
    gan_rot = 1800 #GAN epoch size for initial set
############################################


#preprocessing
    using_dic,asset_list,dic_sedol_as,dic_bench,dic_beta,dic_sector,dic_MCAP,risk_sedol,risk_mat,dic_r,alpha,dic_aseet_MCAP,dic_aseet_sector=preprocessing(date)


#set initial cardinality
    if init_set=='cplex':
        Q_con, qmat, alpha = init_cplex(risk_mat,risk_sedol)
        sol = portfolio(sector=dic_sector,bench=dic_bench,asset=risk_sedol,
                MCAPQ=dic_MCAP,beta=dic_beta,alpha=alpha,qmat=qmat, Q_con=Q_con,multiple=10000,time_init=90)[0][0]
        alpha = []


        for i in risk_sedol:

            alpha.append(-1*dic_sedol_as[i])

    elif init_set=='GAN':

        bw=list(using_dic['bw'])
        bw=np.array(bw)

        beta=list(using_dic['beta'])
        beta=np.array(beta)

        alpha=list(using_dic['as'])
        alpha=np.array(alpha)

        sector_set = set(list(using_dic['sector']))
        sector_list = list(using_dic['sector'])

        gbs=using_dic[['sector', 'index']].apply(tuple, axis=1)
        group_by_sector = [[y[1] for y in gbs if y[0]==x] for x in sector_set]

        mq_list=list(using_dic['mq'])
        mq_set=set(list(using_dic['mq']))

        gbm=using_dic[['mq', 'index']].apply(tuple, axis=1)
        group_by_mq = [[y[1] for y in gbm if y[0]==x] for x in mq_set]

        nbasset=len(bw)
        nbsector=len(sector_set)
        nbmq=len(mq_set)

        best_sols, sols,fea_sample_obj_values,samples=GAN(gan_rot)
       # best,bestsol=GAN_reulst(best_sols, sols,fea_sample_obj_values)

        #print(len(best))


        aa=np.zeros(nbasset)
        for i in range(50):
            sb =samples[i].data.numpy()
            bb=sum(sb>0.0001)
            aa=aa+bb
        general=np.where(aa.max()*0.001<aa)
        good=np.where(aa.max()*0.1<aa)
        plus=list(set(general[0])-set(good[0]))



        best=list(good[0])
        sol={}
        for i in range(len(risk_sedol)):
            if i in best: sol[risk_sedol[i]]=1.
            else: sol[risk_sedol[i]]=0.


    elif init_set=='random':
        np.random.seed()
        r=list(set(list(np.random.randint(len(risk_sedol),size=70))))
        sol={}
        for i in range(len(risk_sedol)):
            if i in r: sol[risk_sedol[i]]=1.
            else: sol[risk_sedol[i]]=0.


# solve & update
    sol=sol
    as_ = using_dic['as']
    if n == 0:
        w_pre = {}
        for j in risk_sedol:
            w_pre.update({j:0})
        results = update(sol,60.,as_,w_pre,0)
            
    else:
        results = update(sol,80.,as_,w_pre,100000)

    result = results[0]
    
    try:
        w_index = result.index(min(result))
        result2 = results[1][w_index]
        print(result2)

        w_pre = w_i_pre_sum(result2)
        w_pre_list.append(w_pre)
        feasi += 1
        list_feasi_obj.append(result)
        print("------objective value 변화------")
        print(result)
#         n += 1
        
    except:
        infeasi += 1
        
    if k == 10:
        fini = False
    k += 1
print("feasible 횟수 : ",feasi)
print("저장된 objective value")
print(list_feasi_obj)
print("infeasible 횟수 : ",infeasi)

0 period
[9, 16, 25, 41, 52, 55, 70, 78, 84, 117, 121, 127, 131, 133, 134, 137, 146, 150, 151, 155, 159, 173, 174, 179, 180, 192, 193, 200, 201, 214, 215, 228, 238, 250, 253, 256, 258, 267, 268, 273, 275, 291, 308, 322, 332, 337, 346, 360, 363, 374, 385, 388, 394, 395, 408, 421, 444, 461, 466, 472, 482, 486, 487]
con
O : 24600000.0
20.926151208738215
AC
0.868814955362
TE
[ 20.93020369]
0.143993234568  wupsum
1.00000084511  wsum!
464  wupdic_len
[ 0.20930204]
0.790697963083
O : 24600000.0
46.480899288923105
TE :  [ 46.47748966]
AS :  0.867551591
Bs infeasible
O : 24600000.0
22.749363370175356
TE :  [ 22.74782084]
AS :  0.868172485392
O : 24600000.0
25.34202259998625
TE :  [ 25.3376668]
AS :  0.86948890969
O : 24600000.0
23.839300704709963
TE :  [ 23.83707075]
AS :  0.868798812384
O : 24600000.0
24.503619683178012
TE :  [ 24.50094223]
AS :  0.869489344828
O : 24600000.0
24.89272862918417
TE :  [ 24.88920904]
AS :  0.869489239051
O : 24600000.0
25.107791147349904
TE :  [ 25.10385525]
AS :

O : 24600000.0
31.992417568211373
TE :  [ 31.99444123]
AS :  0.748115063433
Bs infeasible
O : 24600000.0
28.23157063827648
TE :  [ 28.231052]
AS :  0.745863786751
Bs infeasible
O : 24600000.0
19.295165799246345
TE :  [ 19.29410135]
AS :  0.748765195619
infeasible!
TE infeasible
TE_out :  472
[0, 9, 16, 18, 25, 41, 52, 55, 63, 70, 74, 78, 84, 117, 118, 121, 127, 131, 133, 134, 137, 146, 150, 151, 155, 173, 174, 179, 180, 188, 192, 193, 200, 201, 214, 215, 228, 250, 258, 267, 268, 273, 275, 291, 308, 319, 322, 337, 346, 360, 363, 374, 388, 394, 395, 408, 411, 421, 444, 461, 482, 486, 487]
O : 24600000.0
10.802366334181487
AC
0.738990519774
TE
[ 10.80668776]
0.209171511435  wupsum
1.00000006408  wsum!
467  wupdic_len
[ 0.10806688]
0.891933122394
O : 24600000.0
43.20343788409921
TE :  [ 43.20171357]
AS :  0.742660012849
Bs infeasible
O : 24600000.0
13.075309420925098
TE :  [ 13.07403873]
AS :  0.738766301
O : 24600000.0
17.012333754173593
TE :  [ 17.00856841]
AS :  0.738783051627
O : 24600

[1, 4, 18, 36, 37, 42, 48, 53, 91, 106, 117, 119, 120, 122, 126, 130, 133, 171, 173, 181, 188, 190, 193, 194, 205, 207, 208, 213, 219, 232, 235, 238, 248, 249, 253, 255, 262, 266, 278, 280, 289, 295, 297, 318, 321, 328, 334, 338, 371, 382, 390, 392, 403, 407, 409, 425, 434, 437, 440, 442, 450, 464, 485, 487, 488]
con
O : 24600000.0
24.27005574879759
AC
0.8596100445
TE
[ 24.24179903]
0.230679975226  wupsum
1.00000091684  wsum!
471  wupdic_len
[ 0.24241799]
0.75758200974
O : 24600000.0
57.907352072894135
TE :  [ 57.88067684]
AS :  0.857567018
Bs infeasible
O : 24600000.0
25.590113294523153
TE :  [ 25.56260121]
AS :  0.859702060229
O : 24600000.0
24.58538632537761
TE :  [ 24.55787797]
AS :  0.859677159148
O : 24600000.0
24.984596201771783
TE :  [ 24.957192]
AS :  0.859234058861
O : 24600000.0
25.253700313119488
TE :  [ 25.22631021]
AS :  0.859720519474
O : 24600000.0
25.112529541612272
TE :  [ 25.08512142]
AS :  0.859234095178
BS Feasible!!!
obj_val
25.0741648164
[0, 1, 4, 18, 36, 37, 42,

O : 24600000.0
26.885316910962416
TE :  [ 26.8652673]
AS :  0.828108761441
O : 24600000.0
25.185420299025875
TE :  [ 25.16455525]
AS :  0.828213793428
BS Feasible!!!
obj_val
25.1726154095
[0, 1, 4, 18, 36, 37, 42, 48, 53, 91, 106, 117, 119, 122, 126, 130, 133, 171, 173, 181, 188, 190, 193, 194, 205, 207, 208, 213, 219, 232, 235, 238, 248, 249, 253, 255, 262, 266, 278, 280, 289, 295, 297, 318, 319, 321, 328, 334, 338, 371, 382, 392, 403, 407, 409, 425, 434, 437, 440, 442, 450, 464, 485, 487, 488]
cplex_val
25.185420299025875
O : 24600000.0
17.93299357566295
AC
0.814099539985
TE
[ 17.90784064]
0.260533737077  wupsum
1.00000367714  wsum!
472  wupdic_len
[ 0.17907841]
0.820921593562
O : 24600000.0
59.1849473429654
TE :  [ 59.17602059]
AS :  0.812658022
Bs infeasible
O : 24600000.0
19.50915954449997
TE :  [ 19.48545036]
AS :  0.814498128787
O : 24600000.0
27.904493031150906
TE :  [ 27.88638544]
AS :  0.814497144177
O : 24600000.0
21.897853911404084
TE :  [ 21.8768314]
AS :  0.814497743747
O

[3, 11, 12, 13, 29, 40, 54, 55, 60, 64, 65, 77, 80, 81, 91, 93, 99, 118, 127, 131, 143, 145, 150, 159, 165, 169, 188, 206, 213, 216, 217, 219, 234, 238, 247, 250, 251, 252, 259, 262, 267, 272, 276, 284, 291, 295, 299, 311, 320, 322, 330, 339, 350, 369, 377, 400, 402, 418, 434, 446, 461, 474, 486, 488]
con
O : 24600000.0
16.228137738148185
AC
0.822353600643
TE
[ 16.23446449]
0.271274187506  wupsum
1.00000013742  wsum!
466  wupdic_len
[ 0.16234464]
0.837655355114
O : 24600000.0
22.63556022292701
TE :  [ 22.64196824]
AS :  0.835232692387
O : 24600000.0
85.77613308493729
TE :  [ 85.76750358]
AS :  0.855694391533
Bs infeasible
O : 24600000.0
39.21319996773873
TE :  [ 39.21409217]
AS :  0.840956048254
O : 24600000.0
27.745946173250378
TE :  [ 27.74994005]
AS :  0.837809745822
O : 24600000.0
24.451153983517376
TE :  [ 24.4567176]
AS :  0.836166633558
O : 24600000.0
25.895644951944597
TE :  [ 25.90042094]
AS :  0.837026069075
BS Feasible!!!
obj_val
25.8633882917
[3, 11, 12, 13, 18, 29, 40, 54,

...

O : 24600000.0
16.143043079811868
AC
0.80363581
TE
[ 16.13706702]
0.246332795206  wupsum
1.00000005548  wsum!
468  wupdic_len
[ 0.16137067]
0.838629329752
O : 24600000.0
42.84597877910792
TE :  [ 42.83720462]
AS :  0.804510663379
O : 24600000.0
17.977982940009475
TE :  [ 17.97144315]
AS :  0.804013810281
O : 24600000.0
21.818489645370583
TE :  [ 21.81243246]
AS :  0.80451194995
O : 24600000.0
28.75346928580727
TE :  [ 28.74612201]
AS :  0.804511725821
O : 24600000.0
24.397910118777016
TE :  [ 24.39126702]
AS :  0.804511781309
O : 24600000.0
26.3516809556379
TE :  [ 26.34468499]
AS :  0.804511565371
BS Feasible!!!
obj_val
26.3766779181
[0, 4, 6, 13, 14, 17, 18, 22, 23, 34, 38, 41, 49, 50, 55, 74, 102, 106, 113, 115, 118, 130, 145, 154, 164, 182, 188, 190, 196, 223, 225, 227, 244, 248, 252, 253, 254, 255, 257, 268, 317, 319, 327, 328, 355, 360, 367, 369, 370, 378, 401, 406, 410, 414, 421, 428, 437, 447, 448, 463, 466, 471, 483]
cplex_val
26.3516809556379
O : 24600000.0
11.650712934789258

[0, 3, 5, 28, 32, 37, 70, 79, 95, 106, 111, 115, 127, 128, 138, 152, 153, 176, 179, 198, 215, 217, 228, 233, 235, 239, 243, 255, 276, 280, 287, 292, 293, 295, 298, 299, 306, 308, 314, 334, 337, 342, 343, 347, 349, 351, 370, 396, 399, 407, 413, 418, 422, 426, 436, 458, 465, 468, 469, 471, 480, 489]
con
O : 24600000.0
16.18325567019091
AC
0.864080891
TE
[ 16.19235042]
0.27555847871  wupsum
1.00000008158  wsum!
471  wupdic_len
[ 0.1619235]
0.838076495797
O : 24600000.0
28.186731424944035
TE :  [ 28.20744926]
AS :  0.864080891
Bs infeasible
O : 24600000.0
23.792288506047868
TE :  [ 23.81155088]
AS :  0.864080891
O : 24600000.0
25.280109090025658
TE :  [ 25.29971061]
AS :  0.864080891
Bs infeasible
O : 24600000.0
23.44740568672933
TE :  [ 23.46512985]
AS :  0.864080891
O : 24600000.0
26.07364160404874
TE :  [ 26.09356676]
AS :  0.864080891
Bs infeasible
O : 24600000.0
24.642899326180245
TE :  [ 24.66197799]
AS :  0.864080891
infeasible!
TE infeasible
TE_out :  215
[0, 3, 5, 18, 28, 32, 37, 

O : 24600000.0
14.65378749694526
AC
0.787343111954
TE
[ 14.65395166]
0.277839442012  wupsum
1.00000259877  wsum!
473  wupdic_len
[ 0.14653952]
0.853460483395
O : 24600000.0
96.78354563292885
TE :  [ 96.77961145]
AS :  0.795585209783
Bs infeasible
O : 24600000.0
17.498842759473376
TE :  [ 17.50834434]
AS :  0.787979150357
O : 24600000.0
37.606053609147644
TE :  [ 37.60992383]
AS :  0.787698289039
O : 24600000.0
24.134780112900632
TE :  [ 24.1415085]
AS :  0.787964320014
O : 24600000.0
30.01653011452539
TE :  [ 30.02187602]
AS :  0.787948974053
O : 24600000.0
26.861989414574293
TE :  [ 26.86803093]
AS :  0.787900260197
BS Feasible!!!
obj_val
26.8572598512
[0, 3, 5, 18, 28, 32, 37, 70, 72, 74, 79, 95, 106, 111, 115, 127, 128, 138, 152, 153, 176, 188, 198, 217, 228, 233, 235, 239, 243, 255, 276, 280, 287, 293, 295, 298, 306, 308, 314, 319, 334, 337, 342, 347, 349, 351, 370, 396, 399, 407, 409, 413, 418, 422, 426, 436, 458, 465, 469, 471, 480, 489]
cplex_val
26.861989414574293
O : 24600000.

[4, 8, 20, 22, 23, 37, 44, 50, 60, 73, 92, 101, 107, 112, 115, 123, 129, 134, 142, 145, 147, 155, 159, 169, 180, 187, 198, 201, 202, 220, 221, 233, 234, 256, 259, 261, 267, 272, 288, 292, 297, 312, 321, 322, 331, 342, 360, 361, 365, 366, 369, 374, 384, 387, 389, 391, 400, 426, 434, 439, 440, 478, 481, 483, 484, 485, 487]
con
O : 24600000.0
16.49640959206563
AC
0.848975927
TE
[ 16.49182967]
0.288795962174  wupsum
1.00000012031  wsum!
468  wupdic_len
[ 0.1649183]
0.835081703288
O : 24600000.0
49.78124010185882
TE :  [ 49.78284437]
AS :  0.848975927
Bs infeasible
O : 24600000.0
20.885703783586113
TE :  [ 20.88455217]
AS :  0.849147865942
O : 24600000.0
39.12439299887042
TE :  [ 39.12391796]
AS :  0.848975927
Bs infeasible
O : 24600000.0
32.4845628630897
TE :  [ 32.48279848]
AS :  0.848975927
Bs infeasible
O : 24600000.0
31.84843861656123
TE :  [ 31.8468299]
AS :  0.848975927
Bs infeasible
O : 24600000.0
29.427816839854632
TE :  [ 29.4264972]
AS :  0.849127820946
Bs infeasible
infeasible!


O : 24600000.0
13.825597486333553
AC
0.77419167
TE
[ 13.82115874]
0.307469230393  wupsum
1.00000029368  wsum!
466  wupdic_len
[ 0.13821159]
0.861788412602
O : 24600000.0
59.311390192501754
TE :  [ 59.31813237]
AS :  0.784043785933
Bs infeasible
O : 24600000.0
17.954144242852426
TE :  [ 17.9464643]
AS :  0.774363699177
O : 24600000.0
43.22596917640668
TE :  [ 43.23012804]
AS :  0.781501813388
Bs infeasible
O : 24600000.0
26.207747834761072
TE :  [ 26.20098991]
AS :  0.774363642711
O : 24600000.0
21.274918379431043
TE :  [ 21.26783903]
AS :  0.774363691235
O : 24600000.0
23.402057654835385
TE :  [ 23.39519989]
AS :  0.774363658815
BS Feasible!!!
obj_val
26.2036034397
[0, 4, 8, 18, 20, 22, 23, 37, 44, 50, 60, 73, 74, 92, 101, 107, 112, 115, 123, 134, 145, 147, 155, 159, 169, 180, 187, 188, 198, 201, 202, 220, 221, 233, 234, 256, 259, 261, 267, 288, 292, 297, 319, 321, 322, 331, 342, 360, 361, 365, 366, 369, 384, 387, 389, 391, 400, 426, 434, 439, 440, 478, 481, 483, 484, 485, 487]
cplex_v

In [27]:
import matplotlib.pyplot as plt
%matplotlib inline
w_pre_list
# for i in w_pre_list:

#     asset=[]
#     for j in list(i.keys()):
#         if i[j]>0:
#             asset.append(i[j])

#     fig, ax = plt.subplots()

#     bar_locations = np.arange(len(asset))

#     ax.bar(bar_locations,asset)

#     fig.show()
import operator
print(w_pre_list[0])
aa =w_pre_list[0]
sort_w_p = sorted(aa.items(),key=operator.itemgetter(1))
print(sort_w_p)
sort_w_p[len(sort_w_p)-1][0]

{'2295677': 0.021713611567001159, '2778844': 0.028145268941934993, '8066051': 0.0, '2342034': 0.0, '2417868': 0.0, '2034524': 0.0, 'B1Z77F6': 0.0, 'B28V183': 0.0, '2914734': 0.0, '2820563': 0.0077492217305638687, '2795393': 0.0, 'B3KFWW1': 0.0, 'B3Q2FJ4': 0.0, 'B40K911': 0.0, 'B4NH4W5': 0.0, 'B5BKPQ4': 0.0, 'B5BT0K0': 0.0051968490675561568, 'B633030': 0.0, 'B63QTN2': 0.02384201628038279, 'B6TFK30': 0.0, '2779397': 0.0, 'B6WVMH3': 0.0, 'B76FBQ0': 0.0, 'B7QQYV9': 0.0, 'B7VD3F2': 0.0, 'B840261': 0.0025058848844675291, 'B84LSN3': 0.0, 'B8DMK08': 0.0, 'B8KQN82': 0.0, 'B92RRW2': 0.0, '2018175': 0.0, '2422442': 0.0, 'B9M2WX3': 0.0, 'BBM4S68': 0.0, '2417987': 0.0, '2162845': 0.0, '2963899': 0.0, 'BD5JS97': 0.0, 'BD95VZ8': 0.0, 'BDHLTQ5': 0.0, '2824770': 0.0, 'BF0FM92': 0.0033851953550397, 'BF0HZC2': 0.0, 'BF6V3T1': 0.0, 'BFG3KF2': 0.0, '2320524': 0.0, '2575896': 0.0, '2983604': 0.0, '2605555': 0.0, 'BJWG213': 0.0, '2767165': 0.0, '2101503': 0.0, 'BLNN369': 0.0031566917415751829, 'BMH4NJ8': 0.0

'2297907'

In [14]:
chchlist = []
sumsum = 0
cadcad = 0
for i in range(len(list_feasi_obj)):
    for j in range(len(list_feasi_obj[i])):
        chchlist.append(list_feasi_obj[i][j])
        sumsum += list_feasi_obj[i][j]
        cadcad += 1

print(min(chchlist))
print(max(chchlist))
print(sumsum/cadcad)

24.999601253230647
42.239238256905836
26.188110514068207


Evaluation Criteria

In [15]:
def w_i_pre_sum(w_dic):
   
    w_pre = 0  
    
    for i in w_dic.keys():


        w_pre += w_dic[i]*(dic_r[i]+1)
     
        
    w_i_pre_dic = {}

    for i in w_dic.keys():
        w_i_pre_dic.update({i:(float(w_dic[i])*(dic_r[i]+1))/w_pre})
#         print((float(w_dic[i])*(dic_sedol_as[i]+1))/w_pre)

    
    return w_i_pre_dic




def turnover(w_dic,w_dic2):
    pre_list = set(list(w_dic.keys()))

    rotp_t0 = 0

    for i in risk_sedol:
        rotp_t0 += w_dic[i]*dic_r[i]
        
    w_i_pre = {}
    
    w_pre = 0
    
    for i in risk_sedol:
        w_pre += w_dic[i]*(dic_r[i]+1)

        
        
    w_i_pre_sum = 0

    for i in risk_sedol:
        w_i_pre.update({i:(w_dic[i]*(dic_r[i]+1))/w_pre})
        w_i_pre_sum += (w_dic[i]*(dic_r[i]+1))/w_pre

        

   
        
        
    pre_list2 = set(list(w_dic2.keys()))

    turnover = 0

    add_key_1 = pre_list-pre_list2
    add_key_2 = pre_list2-pre_list


    for i in add_key_2:
        w_dic.update({i:0})

    for i in add_key_1:
        w_dic2.update({i:0})

    pre_list.union(pre_list2)

    for i in pre_list:
        turnover += abs(w_dic2[i]-w_i_pre[i])

    rOTP = 0

    for i in dic_sedol_as.keys():
        rOTP += w_dic2[i]*dic_sedol_as2[i]

    r_Txadj_t = rOTP - turnover*0.5
    
    result=[]
    result.append(turnover,rOTP,r_Txadj_t)
    result.append(w_i_pre_sum)
    
    print("turnover : " + str(turnover))
    print("r_OTP : " + str(rOTP))
    print("r_Txadj_t : " + str(r_Txadj_t))
    
    return result


def IR_TE(result,bench_t):
    cal1 = 1
    cal2 = 1
    list_std=np.array([])

    for i in range(len(result)):
        cal1 = cal1*(1+result[i])
        cal2 = cal2*(1+bench_t[i])
        list_std = np.append(list_std,resultp[i]-bench_t[i])
    
    IR = (cal1-cal2)/np.std(list_std)
    
    TE = math.sqrt(13)*np.std(list_std)
    
    result = []
    
    result.append(IR,TE)
    
    return result

def R_SR(result,bench,n):
    R_CUM = 1
    B_CUM = 1
    for i in range(len(result)):
        R_CUM = R_CUM*(1+result[t])
        
    for i in range(len(bench)):
        B_CUM = B_CUM*(1+bench[t])
    
    R_CUM = R_CUM-1
    
    B_CUM = B_CUM-1
    
    R_ANN = (1+R_CUM)**(13/n) - 1
    
    B_ANN = (1+B_CUM)**(13/n) - 1
    
    R_EXAN = R_ANN-B_ANN
    
    r_t = np.array(result)
    
    SR = R_CUM/np.std(r_t)
    
    

함수정의 ipp 적용

In [45]:
def cal_obj(sol, risk_mat, as_) :
    a = np.full((len(risk_mat[0]), len(risk_mat[0])), 10000)
    risk_mat = risk_mat * a
    s_rr = np.dot(sol, risk_mat)
    s_r = np.dot(s_rr, sol)
    s_a = np.dot(sol, as_*10000)
    obj = s_r-s_a
    return obj


@ipp.require(cal_obj)
def select_candi(sol) :
    
    diff_sub = np.array([0]*len(as_), float)
    diff_add = np.array([0]*len(as_), float)
    candi = []

    for i in range(len(sol)) :
        if(sol[i] > 0) :
            w = sol[i]
            sol[i] = 0
            diff_sub[i] = cal_obj(sol, risk_mat, as_)
            return diff_sub[i]
            if min(diff_sub[diff_sub>0]) == diff_sub[i] :
                for j in range(len(sol)) :
                    if sol[j] == 0 :
                        sol[j] = w
                        diff_add[j] = cal_obj(sol, risk_mat, as_)
                        sol[j] = 0
            sol[i] = w
        
    sol[np.where(diff_sub == min(diff_sub[diff_sub>0]))[0][0]] = 0
    sol[np.where(diff_add == min(diff_add[diff_add>0]))[0][0]] = 0.1

    for val in range(len(sol)) :
        if sol[val] > 0. :
            candi.append(val)

    return candi


@ipp.require(select_candi)
def update(sol) :
    start_time = time.time()
    new_list = []
    con = []
    nums = 0
    for k in risk_sedol :
        new_list.append(sol[k])
        if sol[k] == 1:
            con.append(nums)
        nums += 1
    result_list = []
    result_list2 = []
    
    while(True):
        if time.time() - start_time > lim_time:
            break
        
        ex.set_con(cons=con)
        new_list2 = []
        global new_list3
        new_list3 = []
        try:
            aabbaa = ex.solve(0.01)
            if aabbaa != 0:
                aaaa = aabbaa[0]
                bbbb= aabbaa[1]
                for k in risk_sedol :
                    new_list2.append(aaaa[k])
                for k in risk_sedol :
                    new_list3.append(bbbb[k])
                new_list = new_list2
                result_list.append(aabbaa[2])
                result_list2.append(aabbaa[0])
                con = select_candi(new_list)
            else:
                print("TE")
#                 con = select_candi(new_list)
        except:
            print("NO")
#             con = select_candi(new_list)
    return result_list,result_list2


def sample_Z(m, n):
    return Variable(torch.Tensor(np.random.uniform(0., 1., size=[m, n])))

        
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / np.sqrt(in_dim / 2.)
    return np.random.normal(size=size, scale=xavier_stddev)


class Generator(nn.Module):
    def __init__(self,  in_dim=10, hd_dim=50, out_dim=500):
        super(Generator, self).__init__()
        
        self.G_W1 = nn.Parameter(torch.from_numpy(xavier_init([in_dim, hd_dim])).float())
        self.G_b1 = nn.Parameter(torch.from_numpy(np.zeros(shape=[hd_dim])).float())
        
    
        self.G_W12 = nn.Parameter(torch.from_numpy(xavier_init([hd_dim, hd_dim])).float())
        self.G_b12 = nn.Parameter(torch.from_numpy(np.zeros(shape=[hd_dim])).float())
        
        self.G_W13 = nn.Parameter(torch.from_numpy(xavier_init([hd_dim, hd_dim])).float())
        self.G_b13 = nn.Parameter(torch.from_numpy(np.zeros(shape=[hd_dim])).float())

        
        self.G_W2 = nn.Parameter(torch.from_numpy(xavier_init([hd_dim, out_dim])).float())
        self.G_b2 = nn.Parameter(torch.from_numpy(np.zeros(shape=[out_dim])).float())

    def forward(self, x):
        x = F.relu( x @ self.G_W1 + self.G_b1)
        x = F.relu( x @ self.G_W12 + self.G_b12)
        x = F.relu( x @ self.G_W13 + self.G_b13)
        x = x @ self.G_W2 + self.G_b2

        return F.sigmoid(x*0.1)


class Discriminator(nn.Module):
    def __init__(self, in_dim=500,train=False):
        super(Discriminator, self).__init__()
        self.tot_infs = 0.0
        self.tot_infs_nor = 0.0
        self.train = train
        self.x_threshold = 0.001
        self.inf_scale = 100.0
        self.Zstar = 1000
        
        self.in_dim = in_dim

        np.random.seed(0)
        self.Omega = Variable(torch.from_numpy(risk_mat)).float()

        self.card_upper = 70.
        self.card_lower = 69.
        
        
    def forward(self, x):
        tot_infs = Variable(torch.zeros(x.size()[0]))
        x_bw = Variable(torch.from_numpy(bw).float())
        alp = Variable(torch.Tensor(alpha).float())   
        bet = Variable(torch.Tensor(beta).float())
 
        # (4)
        tot_infs += F.relu(1. - torch.sum(x,1))
        tot_infs += F.relu(torch.sum(x,1) - 1.)
        
        # (5)
        tot_infs += F.relu(torch.abs(x-x_bw)@ Variable(torch.ones(self.in_dim))-0.05)

        # (6)
        for j in range(nbsector):
            k= (x-x_bw) @ Variable(
                torch.Tensor([1 if i in group_by_sector[j] else 0 for i in range(self.in_dim)]).float())  #sum
            l = torch.abs(k)
            tot_infs += F.relu(l - 0.1)
            
        # (7)
        for j in range(nbmq):
            k = (x-x_bw) @ Variable(
                torch.Tensor([1 if i in group_by_mq[j] else 0 for i in range(self.in_dim)]).float())  #sum
            l = torch.abs(k)
            tot_infs += F.relu(l - 0.1)
            
        # (8)
        k = (x - x_bw)@bet
        l = torch.abs(k)
        tot_infs += F.relu(l - 0.1)
        
        # (9) lb <= card(x) <= b
        if self.train:
            num_non_zeros = torch.sum(torch.tanh(x / self.x_threshold), 1)
            tot_infs += F.relu(num_non_zeros - self.card_upper) + F.relu(self.card_lower - num_non_zeros)
        else:
            num_non_zeros = torch.sum(x.data>self.x_threshold, 1).float()
            tot_infs += Variable(np.maximum(num_non_zeros - self.card_upper, 0) + np.maximum(self.card_lower - num_non_zeros, 0))
    
        
        # (10)
        l = 0.5*torch.sum(torch.abs(x - x_bw))
        tot_infs += F.relu(0.6 - l)
        #tot_infs += F.relu(l - 1.)
    
        
        # (11) a<=dQd <= b
        dQ = (x-x_bw) @ self.Omega
        size_x = x.size()
        dQd = torch.bmm(dQ.view(size_x[0],1,size_x[1]), (x-x_bw).view(size_x[0],size_x[1],1)).squeeze(1).squeeze(1)
        tot_infs += F.relu(dQd - 0.01)
        tot_infs += F.relu(0.0025 - dQd)
        
        
        # min dQd + ad (<= Zstar)
        l = (x-x_bw)@alp

        MONTH =0
        turnover=x_bw
        
        if MONTH > 0 :
            tot_infs += (F.relu(100.*dQd - 100.*l -0.5*turnover - self.Zstar))
            self.objs = 100.*dQd - 100.*l -0.5*turnover
        else:
            tot_infs += (F.relu(50.*dQd - 50.*l  - 0.5*self.Zstar ))
            self.objs = 100.*dQd - 100.*l
        
        
        fea_probs = F.relu(1. - F.tanh(tot_infs/self.inf_scale))  

        
        return fea_probs
        


def GAN(rotation):
    X_dim = nbasset
    HL_dim = 50
    Z_dim = 10

    G = Generator(Z_dim, HL_dim, X_dim)

    sample_D = Discriminator(X_dim, train=False)
    train_D = Discriminator(X_dim, train=True)

    G_solver = optim.Adam(G.parameters(), lr=2e-3)

    mb_size = 128

    best_obj = 100000.0
    best_sols = []

    losses = []

    start_time = time.time()

    feasible_threshold = 0.95

    adaptive_inf_scale = float(nbasset)

    for it in range(rotation):
        if it % 900 == 0:
            sample_noise = sample_Z(128, Z_dim)
            samples = G(sample_noise)

            sample_D.Zstar = best_obj
            sample_D.inf_scale = adaptive_inf_scale

            sample_D_values = sample_D(samples).data.numpy()
            sample_obj_values = sample_D.objs.data.numpy()

            fea_sample_obj_values = sample_obj_values[sample_D_values > feasible_threshold]
            sols = samples.data.numpy()[sample_D_values > feasible_threshold]

            if len(fea_sample_obj_values > 0):
                best_idx = np.argmin(fea_sample_obj_values)
                if np.min(fea_sample_obj_values) < best_obj:
                    best_obj = np.minimum(fea_sample_obj_values[best_idx], best_obj)
                    best_sols.append(sols[best_idx])

        z = Variable(torch.randn(mb_size, Z_dim))

        train_D.Zstar = best_obj
        train_D.inf_scale = adaptive_inf_scale

        G_loss = -torch.mean(torch.log(train_D(G(z))))

        G.zero_grad()
        G_loss.backward()
        G_solver.step()
        
    sam=[]
    for i in range(50):
        n = sample_Z(128, 10)
        samples = G(n)
        sam.append(samples)
        
    return best_sols, sols,fea_sample_obj_values,sam

        

def GAN_reulst(best_sols, sols,fea_sample_obj_values):    
    good_sols=np.argsort(fea_sample_obj_values)[:10]
    card_list=[list(np.where(sols[i]>0.001)[0]) for i in range(len(good_sols))]
    card_freq=np.zeros(nbasset)
    for i in range(len(card_list)):
        for j in card_list[i]:
            card_freq[j]+=1
    best=np.argsort(sum(sols))[::-1][:len(card_freq[card_freq>len(card_list)-5])]
    bestsol=np.where(best_sols[0]>0.001)[0]
    return best,bestsol

def init_cplex(risk_mat,risk_sedol):
    sedol_var_list =[]
    for i in risk_sedol:
        sedol_var_list.append("d"+str(i))

    for i in risk_sedol:
        sedol_var_list.append("q"+str(i))

    sedol_var_list.append("assum")
    #global alpha
    alpha = []
    for i in risk_sedol:
        alpha.append(-10000.*dic_sedol_as[i])

    qmat=[]
    for i in range(len(risk_mat)):
        qmat_1=[]
        qmat_1.append(sedol_var_list)
        new_risk_mat=[]
        for j in risk_mat[i]:
            new_risk_mat.append(20000*j)
        for j in risk_mat[i]:
            new_risk_mat.append(0)
        new_risk_mat.append(0)
        qmat_1.append(new_risk_mat)
        qmat.append(qmat_1)

    for i in range(len(risk_mat)):
        qmat_1=[]
        qmat_1.append(sedol_var_list)
        new_risk_mat=[]
        for j in risk_mat[i]:
            new_risk_mat.append(0)
        for j in risk_mat[i]:
            new_risk_mat.append(0)
        new_risk_mat.append(0)
        qmat_1.append(new_risk_mat)
        qmat.append(qmat_1)

    for i in range(1):
        qmat_1=[]
        qmat_1.append(sedol_var_list)
        new_risk_mat=[]
        for j in risk_mat[i]:
            new_risk_mat.append(0)
        for j in risk_mat[i]:
            new_risk_mat.append(0)
        new_risk_mat.append(0)
        qmat_1.append(new_risk_mat)
        qmat.append(qmat_1)

    q_con1 = []
    q_con2 = []
    q_val = []


    for i in range(len((risk_mat[0]))):
        for j in range(len(risk_mat[0])):
            if j >= i:
                q_con1.append(i)
                q_con2.append(j)
                if i == j:
                    ex_list = list(risk_mat[i])
                    q_val.append(0.5*ex_list[j]*10000)
                else:
                    ex_list = list(risk_mat[i])
                    q_val.append(ex_list[j]*10000)

    Q_con = []
    Q_con.append(q_con1)
    Q_con.append(q_con2)
    Q_con.append(q_val)
    
    return Q_con, qmat, alpha

print("complete")

complete


solve 적용

In [50]:
#for i in date: 이런식으로 하되 evaluation criteria 는 아직 반영안함..!
feasi = 0
infeasi = 0
list_feasi_obj = []
list_feasi_sol = []

############################################
n=0
date=date_list[n]
init_set='ipp' #initial set by GAN, cplex, random

c_time = 10. #cplex time for initial set
gan_rot = 1800 #GAN epoch size for initial set
############################################

using_dic,asset_list,dic_sedol_as,dic_bench,dic_beta,dic_sector,dic_MCAP,risk_sedol,risk_mat,dic_r,alpha=preprocessing(date)
as_ = using_dic['as']
    
dv['as_'] = as_
dv['w_pre'] = w_pre
dv['dic_sector'] = dic_sector
dv['dic_bench'] = dic_bench
dv['risk_sedol'] = risk_sedol
dv['dic_MCAP'] = dic_MCAP
dv['dic_beta'] = dic_beta
dv['alpha'] = alpha
dv['risk_mat'] = risk_mat
dv['using_dic'] = risk_mat

# solve & update
for k in range(1):
    print("solve print : ", k, "번째")
    #set initial cardinality
    if init_set=='cplex':
        Q_con, qmat, alpha = init_cplex(risk_mat,risk_sedol)
        sol = portfolio(sector=dic_sector,bench=dic_bench,asset=risk_sedol,
                MCAPQ=dic_MCAP,beta=dic_beta,alpha=alpha,qmat=qmat, Q_con=Q_con,multiple=10000,time_init=90)[3][0]
        alpha = []


        for i in risk_sedol:

            alpha.append(-1*dic_sedol_as[i])
    elif init_set=='GAN':

        bw=list(using_dic['bw'])
        bw=np.array(bw)

        beta=list(using_dic['beta'])
        beta=np.array(beta)

        alpha=list(using_dic['as'])
        alpha=np.array(alpha)

        sector_set = set(list(using_dic['sector']))
        sector_list = list(using_dic['sector'])

        gbs=using_dic[['sector', 'index']].apply(tuple, axis=1)
        group_by_sector = [[y[1] for y in gbs if y[0]==x] for x in sector_set]

        mq_list=list(using_dic['mq'])
        mq_set=set(list(using_dic['mq']))

        gbm=using_dic[['mq', 'index']].apply(tuple, axis=1)
        group_by_mq = [[y[1] for y in gbm if y[0]==x] for x in mq_set]

        nbasset=len(bw)
        nbsector=len(sector_set)
        nbmq=len(mq_set)

        best_sols, sols,fea_sample_obj_values,samples=GAN(gan_rot)
       # best,bestsol=GAN_reulst(best_sols, sols,fea_sample_obj_values)

        #print(len(best))


        aa=np.zeros(nbasset)
        for i in range(50):
            sb =samples[i].data.numpy()
            bb=sum(sb>0.0001)
            aa=aa+bb
        general=np.where(aa.max()*0.001<aa)
        good=np.where(aa.max()*0.1<aa)
        plus=list(set(general[0])-set(good[0]))



        best=list(good[0])
        sol={}
        for i in range(len(risk_sedol)):
            if i in best: sol[risk_sedol[i]]=1.
            else: sol[risk_sedol[i]]=0.


#         best1=list(good[0])+list(np.random.choice(plus, np.random.randint(3,5)))
#         sol1={}
#         for i in range(len(risk_sedol)):
#             if i in best1: sol1[risk_sedol[i]]=1.
#             else: sol1[risk_sedol[i]]=0.

#         best2=list(good[0])+list(np.random.choice(plus, np.random.randint(3,5)))        
#         sol2={}
#         for i in range(len(risk_sedol)):
#             if i in best2: sol2[risk_sedol[i]]=1.
#             else: sol2[risk_sedol[i]]=0.

#         best3=list(good[0])+list(np.random.choice(plus, np.random.randint(3,5)))        
#         sol3={}
#         for i in range(len(risk_sedol)):
#             if i in best3: sol3[risk_sedol[i]]=1.
#             else: sol3[risk_sedol[i]]=0.

#         best4=list(good[0])+list(np.random.choice(plus, np.random.randint(3,5)))        
#         sol4={}
#         for i in range(len(risk_sedol)):
#             if i in best4: sol4[risk_sedol[i]]=1.
#             else: sol4[risk_sedol[i]]=0.

#         best5=list(good[0])+list(np.random.choice(plus, np.random.randint(3,5)))        
#         sol5={}
#         for i in range(len(risk_sedol)):
#             if i in best5: sol5[risk_sedol[i]]=1.
#             else: sol5[risk_sedol[i]]=0.       

#         best6=list(good[0])+list(np.random.choice(plus, np.random.randint(3,5)))                
#         sol6={}
#         for i in range(len(risk_sedol)):
#             if i in best6: sol6[risk_sedol[i]]=1.
#             else: sol6[risk_sedol[i]]=0.       

#         best7=list(good[0])+list(np.random.choice(plus, np.random.randint(3,5)))                
#         sol7={}
#         for i in range(len(risk_sedol)):
#             if i in best7: sol7[risk_sedol[i]]=1.
#             else: sol7[risk_sedol[i]]=0.

    elif init_set=='random':
        r=list(set(list(np.random.randint(len(risk_sedol),size=70))))
        sol={}
        for i in range(len(risk_sedol)):
            if i in r: sol[risk_sedol[i]]=1.
            else: sol[risk_sedol[i]]=0.
    
    elif init_set=='ipp':
        sol = {'2295677': 0.0, '2778844': 0.0, '8066051': 1.0, '2342034': 0.0, '2417868': 1.0, '2034524': 0.0, 'B1Z77F6': 0.0, 'B28V183': 0.0, '2914734': 0.0, '2820563': 1.0, '2795393': 0.0, 'B3KFWW1': 1.0, 'B3Q2FJ4': 0.0, 'B40K911': 0.0, 'B4NH4W5': 0.0, 'B5BKPQ4': 1.0, 'B5BT0K0': 0.0, 'B633030': 0.0, 'B63QTN2': 0.0, 'B6TFK30': 0.0, '2779397': 0.0, 'B6WVMH3': 1.0, 'B76FBQ0': 0.0, 'B7QQYV9': 0.0, 'B7VD3F2': 0.0, 'B840261': 0.0, 'B84LSN3': 0.0, 'B8DMK08': 0.0, 'B8KQN82': 0.0, 'B92RRW2': 0.0, '2018175': 0.0, '2422442': 0.0, 'B9M2WX3': 0.0, 'BBM4S68': 1.0, '2417987': 0.0, '2162845': 0.0, '2963899': 0.0, 'BD5JS97': 0.0, 'BD95VZ8': 0.0, 'BDHLTQ5': 1.0, '2824770': 0.0, 'BF0FM92': 0.0, 'BF0HZC2': 0.0, 'BF6V3T1': 0.0, 'BFG3KF2': 1.0, '2320524': 0.0, '2575896': 0.0, '2983604': 0.0, '2605555': 0.0, 'BJWG213': 0.0, '2767165': 0.0, '2101503': 0.0, 'BLNN369': 0.0, 'BMH4NJ8': 0.0, '2819118': 0.0, '2692632': 0.0, 'BSPHGL4': 1.0, '2331225': 0.0, 'BTN1Y11': 0.0, 'BTN1Y44': 0.0, '2522096': 1.0, 'BVFMFG8': 0.0, 'BWSW5J5': 0.0, '2367480': 1.0, 'BY7QL61': 0.0, 'BY9D546': 0.0, 'BYND5N1': 0.0, 'BYND5W0': 0.0, 'BYNF418': 1.0, 'BYP66M4': 0.0, 'BYQQ3P5': 0.0, 'BYV2325': 0.0, 'BYVY8G0': 0.0, 'BYWTW73': 1.0, 'BYX4D52': 0.0, 'BYXB201': 0.0, 'BYXD7B3': 0.0, 'BYZGYC0': 1.0, 'BZ0P3Z5': 0.0, 'BZ2Y3W7': 0.0, 'BZBY209': 0.0, '2520153': 0.0, '2769796': 0.0, '2341484': 0.0, '2624486': 0.0, '2252058': 0.0, '2517382': 1.0, '2714923': 0.0, '2880255': 0.0, '2023737': 0.0, '2210614': 1.0, '2017213': 0.0, '2218447': 0.0, '2154927': 0.0, '2884224': 0.0, '2511908': 0.0, '2319157': 0.0, '2816409': 0.0, '2654320': 0.0, '2229676': 0.0, '2429090': 0.0, '2588184': 0.0, '2648806': 0.0, '2098876': 0.0, '2903550': 0.0, '2216850': 0.0, '2516839': 0.0, '2246288': 0.0, '2599142': 0.0, '2367026': 0.0, '2933632': 1.0, '2965839': 0.0, '2477152': 0.0, '2026082': 0.0, '2818740': 0.0, '2365161': 0.0, '2681511': 0.0, 'B198391': 0.0, '2588173': 0.0, '2790611': 0.0, '2008154': 0.0, '2047317': 0.0, '2185046': 0.0, '2804211': 0.0, '2885937': 0.0, '2215460': 0.0, '2235822': 1.0, '2550161': 0.0, '2936921': 0.0, '2041364': 1.0, '2466321': 0.0, '2707677': 0.0, '2434209': 0.0, '2885700': 0.0, '2550707': 0.0, '2476193': 0.0, '2195722': 0.0, '2632650': 0.0, '2236911': 1.0, '2880501': 0.0, '2261526': 0.0, '2803014': 1.0, '2480138': 1.0, '2365804': 0.0, '2871505': 1.0, '2073408': 0.0, '2066721': 0.0, '2034494': 0.0, '2295172': 0.0, '2649100': 0.0, '2261203': 0.0, '2758051': 0.0, '2113434': 0.0, '2110703': 0.0, '2769503': 0.0, '2578516': 0.0, '2967181': 0.0, '2038430': 0.0, '2639509': 0.0, '2280220': 0.0, '2681403': 1.0, '2570200': 0.0, '2077905': 1.0, '2717320': 0.0, '2511920': 0.0, '2767381': 0.0, '2465254': 0.0, '2232793': 1.0, '2909730': 0.0, '2346508': 0.0, '2842255': 0.0, '2665861': 0.0, '2829601': 0.0, '2126335': 0.0, '2183244': 0.0, '2767228': 1.0, '2039831': 0.0, '2284761': 0.0, 'B1FLZ21': 0.0, '2407966': 1.0, '2670519': 0.0, '2807566': 0.0, '2692397': 1.0, '2142784': 1.0, '2085878': 0.0, '2232696': 0.0, '2142922': 0.0, '2485070': 0.0, '2326618': 0.0, '2705024': 1.0, '2516378': 0.0, '2661568': 0.0, '2019479': 0.0, '2336747': 0.0, 'B067BM3': 0.0, '2672689': 0.0, '2378200': 0.0, '2101967': 0.0, '2182553': 1.0, '2883395': 0.0, 'B1F76F9': 1.0, '2684703': 0.0, '2577609': 0.0, '2214832': 0.0, '2634805': 0.0, 'B198380': 0.0, 'B01R311': 1.0, '2614807': 0.0, '2413758': 0.0, '2942188': 0.0, '2272205': 0.0, '2692665': 1.0, '2445966': 0.0, '2635701': 0.0, '2295945': 0.0, '2198163': 0.0, '2915500': 0.0, '2496113': 1.0, '2270726': 1.0, '2457552': 0.0, '2928683': 0.0, '2705648': 1.0, '2344331': 0.0, '2985677': 1.0, '2345022': 0.0, '2580986': 0.0, '2094670': 0.0, '2259468': 1.0, '2422806': 0.0, '2626921': 0.0, '2931205': 0.0, '2698470': 0.0, '2668804': 0.0, '2671501': 0.0, '2428008': 0.0, '2299011': 0.0, '2510723': 1.0, '2256511': 0.0, '2842040': 1.0, '2313405': 0.0, '2867913': 0.0, '2068921': 0.0, '2695921': 0.0, '2279303': 0.0, '2304227': 0.0, '2289874': 0.0, '2685717': 0.0, '2202536': 0.0, '2861078': 0.0, '2960384': 0.0, '2043694': 1.0, '2484088': 0.0, '2654461': 0.0, '2542049': 0.0, '2615468': 0.0, '2459020': 0.0, '2020459': 0.0, '2378534': 1.0, '2937689': 0.0, '2209106': 0.0, '2548616': 0.0, '2886907': 0.0, '2085102': 0.0, '2859954': 0.0, '2405302': 0.0, '2464972': 0.0, '2892090': 0.0, '2367598': 0.0, '2624356': 0.0, '2497406': 1.0, '2044545': 0.0, '2057059': 0.0, '2838555': 0.0, '2471789': 0.0, '2005973': 0.0, '2090173': 1.0, '2696838': 1.0, '2954019': 0.0, '2149309': 0.0, '2204026': 1.0, '2352118': 0.0, '2098597': 0.0, '2852533': 0.0, '2760669': 1.0, '2655408': 0.0, '2822600': 0.0, '2831543': 0.0, '2896713': 0.0, '2646015': 0.0, '2685007': 0.0, '2645409': 0.0, '2829515': 0.0, '2190385': 0.0, 'B011WL6': 0.0, '2536763': 0.0, '2220884': 0.0, '2699291': 0.0, '2192295': 0.0, '2680905': 0.0, '2910970': 0.0, '2160753': 0.0, '2333889': 0.0, '2305844': 0.0, '2765802': 0.0, '2589198': 0.0, '2250870': 0.0, '2319146': 0.0, '2941925': 1.0, '2814641': 0.0, '2445063': 0.0, '2269768': 0.0, '2958936': 0.0, '2069065': 0.0, '2884183': 0.0, '2573209': 0.0, '2853688': 0.0, '2922544': 0.0, '2032067': 0.0, '2732635': 0.0, '2380498': 0.0, '2636607': 0.0, 'B1L60G9': 0.0, '2377809': 0.0, '2011602': 1.0, '2310194': 0.0, '2249629': 0.0, '2650748': 0.0, '2108601': 0.0, '2510682': 0.0, '2086138': 0.0, '2766805': 0.0, '2073022': 0.0, '2026361': 0.0, '2463247': 0.0, '2611206': 0.0, '2680303': 0.0, '2625047': 0.0, '2032380': 1.0, '2023748': 0.0, '2937667': 0.0, '2831811': 0.0, '2440637': 0.0, '2002479': 0.0, '2002305': 0.0, '2963372': 0.0, '2105505': 0.0, '2630643': 0.0, '2491594': 0.0, '2641827': 0.0, '2350684': 0.0, '2050832': 0.0, '2019952': 0.0, '2028572': 0.0, '2523044': 0.0, 'B0SRLH6': 0.0, '2379504': 0.0, '2547044': 0.0, '2594437': 0.0, '2087807': 0.0, '2021690': 0.0, '2702791': 0.0, '2736035': 0.0, '2046552': 0.0, '2570545': 0.0, '2081779': 0.0, '2224701': 0.0, '2170473': 0.0, '2681801': 0.0, '2024020': 0.0, '2595708': 1.0, '2917766': 0.0, '2182779': 0.0, '2046251': 0.0, '2754060': 0.0, '2090571': 0.0, '2048804': 0.0, '2257019': 0.0, '2000019': 0.0, '2369174': 0.0, '2786159': 0.0, '2572303': 0.0, '2812452': 0.0, '2567741': 0.0, '2704407': 1.0, '2702337': 0.0, '2262314': 1.0, '2065308': 0.0, '2717483': 0.0, '2455965': 0.0, '2180201': 0.0, '2293819': 0.0, '2027342': 0.0, '2212870': 0.0, '2138158': 0.0, '2380863': 0.0, '2885409': 0.0, '2632003': 0.0, '2026242': 0.0, '2490911': 0.0, '2779201': 0.0, '2206657': 1.0, '2007849': 0.0, '2360326': 1.0, '2175672': 0.0, '2109002': 0.0, '2084637': 0.0, '2576350': 0.0, '2029809': 0.0, '2881537': 0.0, '2297907': 1.0, '2947956': 0.0, '2475833': 0.0, '2023607': 0.0, '2783815': 0.0, '2498993': 0.0, '2526117': 0.0, 'B0F0F09': 0.0, '2510615': 0.0, '2240202': 0.0, '2197137': 0.0, '2486813': 0.0, '2411138': 0.0, '2219224': 0.0, 'B0J7D57': 0.0, '2411053': 0.0, '2892045': 0.0, '2419240': 0.0, '2066408': 0.0, 'B08G9Z7': 0.0, '2480677': 0.0, '2641894': 1.0, '2868165': 0.0, '2312736': 0.0, '2100920': 0.0, '2065955': 0.0, '2651086': 0.0, '2511328': 0.0, 'B142VX8': 0.0, '2689560': 0.0, '2340168': 0.0, '2404569': 0.0, '2931595': 0.0, '2043962': 0.0, '2650793': 0.0, '2708841': 0.0, '2328915': 0.0, '2300601': 0.0, 'B00PWV9': 0.0, '2989301': 0.0, '2250687': 0.0, '2746829': 1.0, '2278407': 0.0, '2523022': 0.0, '2183330': 0.0, '2516152': 0.0, '2476803': 0.0, '2464165': 0.0, '2431846': 0.0, '2318024': 0.0, '2414580': 1.0, '2033004': 0.0, '2433842': 0.0, '2196888': 1.0, '2640147': 0.0, '2298386': 0.0, '2279002': 0.0, '2317087': 0.0, '2065159': 0.0, '2259101': 0.0, '2604206': 0.0, '2807492': 0.0, '2701271': 0.0, '2491839': 1.0, '2146838': 0.0, '2341860': 0.0, '2860990': 0.0, 'B12GHV2': 0.0, 'B0SRLF4': 0.0, '2648668': 0.0, '2841489': 1.0, '2655981': 0.0, '2334150': 0.0, '2690506': 0.0, '2182348': 1.0, '2199326': 0.0, '2196479': 0.0, '2859868': 0.0, '2989828': 0.0, '2586122': 0.0, '2674458': 0.0, '2830904': 0.0, '2027104': 0.0}
        sol_list = [sol]*8
        dv.execute('lim_time = 80.')
        
    if n == 0:
        w_pre = {}
        for j in risk_sedol:
            w_pre.update({j:0})
        
        dv.execute('O_scale = 0')
        dv.execute('ex = inform.informs(10000, 10000, w_pre, O_scale,dic_sector,dic_bench,risk_sedol,dic_MCAP,dic_beta,alpha,risk_mat)')
        dv.execute(' ex.set_omega(risk_mat)')
        results = dv.map(update, sol_list)
            
    else:
        dv.execute('O_scale = 1')
        dv.execute('ex = inform.informs(10000, 10000, w_pre, O_scale,dic_sector,dic_bench,risk_sedol,dic_MCAP,dic_beta,alpha,risk_mat)')
        dv.execute(' ex.set_omega(risk_mat)')
        results = dv.map(update, sol_list)
    
    print(results)
    
    for i in range(8) :
        for j in range(len(results[i][0])) :
            list_feasi_obj.append(results[i][0][j])
            list_feasi_sol.append(results[i][0][j])
    try:
        w_index = result.index(min(result))
        result2 = results[1][w_index]
        print(result2)

        w_pre = w_i_pre_sum(result2)
        feasi += 1
        list_feasi_obj.append(result)
        print("------objective value 변화-------")
        print(result)
        
    except:
        infeasi += 1

        
print("solve print : feasible 횟수 : ",feasi)
print("solve print : infeasible 횟수 : ",infeasi)
print("저장된 objective value")
print(list_feasi_obj)
print("저장된 solution")
print(list_feasi_sol)

solve print :  0 번째
[([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], [])]
solve print : feasible 횟수 :  0
solve print : infeasible 횟수 :  1
저장된 objective value
[]
저장된 solution
[]
